In [61]:
# Script: calculate time difference between ros file & video

In [62]:
import pandas as pd
import os
import regex as re
import time


In [63]:
participant = 5
data_path = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/ROS/p'+str(participant) +'/Interm'
out = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/ROS/p'+str(participant) +'/Interm/p'+str(participant) +'_ros_timediff.csv'

In [64]:
# Lookup file: contains video filenames 

file = 'session_map.csv'
lookup = pd.read_csv(file)
lookup = lookup[lookup['participant']==participant]
lookup.head()

,participant,session,filename
0,5,2,webcam_test_2018-04-09-20-43-15
1,5,7,webcam_test_2018-04-18-17-37-42
2,5,8,webcam_test_2018-04-22-17-47-40
3,5,10,webcam_test_2018-04-25-20-34-39
4,5,13,webcam_test_2018-05-06-16-17-51


In [65]:
answer = pd.DataFrame(columns=['file', 'time_diff'])

In [66]:
for subdir, dirs, files in os.walk(data_path):
    for file in files:
        if 'ros' in file and 'csv' in file:
            if 'timediff' in file:
                continue
            # Session for ros file 
            sess_regex1 = 's\d*_'
            sess_regex2 = 's\d*-\d*_'

            res1 = re.findall(sess_regex1, file)
            res2 = re.findall(sess_regex2, file)

            if (res2):
                session = res2[0]
                session = session[1:-1]
            else:
                session = res1[0]
                session = session[1:-1]
            
            # Corresponding video
            l = lookup[lookup['session']==session]
            l = l.reset_index(drop=True)
            if len(l) != 1:
                print('ERROR')
            video = l.loc[0, 'filename']   
                
            # Get Epoch Time of Video File, 
            if 'test' in video:
                video = video.replace('webcam_test_', '')
            elif 'webcam' in video:
                video = video.replace('webcam_', '')
                
            pattern = '%Y-%m-%d-%H-%M-%S'
            video_epoch = int(time.mktime(time.strptime(video, pattern)))

            # Get Epoch Time of Ros File
            temp = pd.read_csv(data_path + '/' + file)           
            ros_epoch = temp['time'][0]
            
            diff = ros_epoch - video_epoch
            answer = answer.append({'file': file, 'time_diff': diff}, ignore_index=True, sort=True)

In [67]:
answer.head()

,file,time_diff
0,p5_s10_ros_2018-04-25-20-34-01_2018-04-25-20-3...,3
1,p5_s7_ros_2018-04-18-17-37-04_2018-04-18-17-37...,8
2,p5_s8_ros_2018-04-22-17-47-02_2018-04-22-17-47...,9
3,p5_s13_ros_2018-05-06-16-17-13_2018-05-06-16-1...,5
4,p5_s2_ros_2018-04-09-20-42-37_2018-04-09-20-42...,6


In [68]:
answer.to_csv(out, index=False)